In [1]:
import pandas as pd
df = pd.read_excel("data/law_清理過3.xlsx")

In [2]:
df.head()

,Id,court,number,date,reason,content,判決
0,726,臺灣台北地方法院 裁判書 -- 民事類,臺灣台北地方法院 103 年婚字第 390 號民事判決,105.02.19,離婚等,臺灣臺北地方法院民事判決103年度婚字第390號原告蕭永崑訴訟代理人連1鴻律師被告彭宜慧訴訟...,駁回
1,727,臺灣台北地方法院 裁判書 -- 民事類,臺灣台北地方法院 104 年婚字第 236 號民事其他,104.10.30,離婚,"臺灣臺北地方法院裁判書--民事類【裁判字號】104,婚,236【裁判日期】1041030【裁...",駁回
2,728,臺灣台北地方法院 裁判書 -- 民事類,臺灣台北地方法院 103 年婚字第 229 號民事判決,104.10.23,離婚等,臺灣臺北地方法院民事判決103年度婚字第229號原告林貞靜被告晏世豪上列當事人間請求離婚等事...,准許
3,729,臺灣台北地方法院 裁判書 -- 民事類,臺灣台北地方法院 103 年婚字第 386 號民事判決,104.04.10,離婚等,臺灣臺北地方法院民事判決103年度婚字第386號原告顏承濬訴訟代理人吳妙白律師被告顏耶咪（E...,准許
4,730,臺灣台北地方法院 裁判書 -- 民事類,臺灣台北地方法院 102 年婚字第 351 號民事其他,104.01.30,離婚等,"臺灣臺北地方法院裁判書--民事類【裁判字號】102,婚,351【裁判日期】1040130【裁...",准許


In [24]:
df['判決'].value_counts()

准許    3156
駁回     526
Name: 判決, dtype: int64

In [3]:
# 一個以上空白還沒處理
df['content'] = df['content'].str.replace(" ","")
df['content'] = df['content'].str.replace("一","1").str.replace("二","2").str.replace("三","3").str.replace("四","4").str.replace("五","5").str.replace("六","6").str.replace("七","7").str.replace("八","8").str.replace("九","9").str.replace("零","0").str.replace("○","0").str.replace("１","1").str.replace("５",'5').str.replace("０",'0')
df['content'] = df['content'].str.replace("千","").str.replace("十","").str.replace("\n","").str.replace("\u3000","").replace('及',"")

In [4]:
df2 = df[df['判決']=="准許"]
df2 = df2.reset_index(drop=True)

In [5]:
import re
regex_new = re.compile('([民同]法第1052條第\d(項|款)((第?\d+(?:、\d+)*款?)|以外之重大事由)?)')

In [7]:
for i in df[df['Id']==726]['content']:
    match = re.findall(regex_new,i)
print(match)

[('民法第1052條第1項第3款', '項', '第3款', '第3款'), ('民法第1052條第1項第3款', '項', '第3款', '第3款'), ('民法第1052條第2項', '項', '', ''), ('民法第1052條第1項第3款', '項', '第3款', '第3款')]


### find_all找尋所有1052法條

In [6]:
law, law_id = [], []
for i,k in list(zip(df2['content'],df2['Id'])):
    law_temp = []
    match_new = re.findall(regex_new,i)
    for a in match_new:
        for b in a:
            if '1052' in b:
                law_temp.append(b)
    law.append(law_temp)
    law_id.append(k)
    

In [7]:
law1052 = list(zip(law, law_id))

In [10]:
law1052[0:3]

[(['民法第1052條第1項第3款', '民法第1052條第1項第3款', '民法第1052條第1項第3款'], 728),
 (['民法第1052條第2項', '民法第1052條第1項以外之重大事由', '民法第1052條第2項', '民法第1052條第2項'], 729),
 (['民法第1052條第1項第2款',
   '民法第1052條第1項第3款',
   '民法第1052條第1項以外之重大事由',
   '民法第1052條第2項',
   '民法第1052條第2項',
   '民法第1052條第2項',
   '民法第1052條第1項第2款',
   '民法第1052條第1項第3款',
   '民法第1052條第2項'],
  730)]

In [11]:
for i in set(law1052[1][0]):
    print(i)

民法第1052條第1項以外之重大事由
民法第1052條第2項


In [60]:
# 先依順序set那一篇出現過的法條
# 再依序split每一個法條，並找尋最後一個被切出的段落
# 在看判准or准許兩詞有無出現在最後一個段落
number = 0
correct, ID = [], []
judge_word = ["判准","准許","自屬有據","即有理由","准予","既為有理由"]
for i,k in list(zip(df2['content'],df2['Id'])):
    
    # 先處理文章，切出判斷字前面的文章，後面的不要，避免後面又出現一樣的法條造成找不到法條
#     article = ''
#     for word in judge_word:
#         if word in i: # 先判斷文章有無此字
#             if len(i.split(word)) > 2:
#                 stop = len(i.split(word))
#                 for i2 in i.split(word):
#                     if stop == 1:
#                         break
#                     else:
#                         article += i2
#                         stop -= 1
#             else:
#                 article += i.split(word)[0]

    # 如果set後只有一條法案，合理推測那一條就是判決准許的法案
    if len(set(law1052[number][0])) == 1:
        correct.append(set(law1052[number][0]))
    elif len(set(law1052[number][0])) > 1:
        # 依序切開法條
        a = []
        for z in set(law1052[number][0]):
            for word in judge_word:
                if len(a) == 1:
                    break
                if word in i.split(z)[-1]: # i是文章，z是法條，如果是True通過
                    a.append(z)
        try:
            correct.append(a[-1])
        except:
            correct.append(None)
    ID.append(k)
    number += 1
final = list(zip(correct, ID))

In [39]:
if len(i.split('准許')) > 2:
    a = ''
    stop = len(i.split('准許'))
    for i2 in i.split('准許'):
        if stop == 1:
            break
        else:
            a += i2
            stop -= 1

In [54]:
final

[({'民法第1052條第1項第3款'}, 728),
 ('民法第1052條第2項', 729),
 ('民法第1052條第1項第3款', 730),
 ('民法第1052條第1項第3款', 731),
 ('民法第1052條第2項', 732),
 ('民法第1052條第1項第3款', 733),
 ('民法第1052條第2項', 735),
 ('民法第1052條第2項', 736),
 ('民法第1052條第1項第3款', 737),
 ('民法第1052條第1項第3款', 738),
 ('民法第1052條第1項第3款', 739),
 ('民法第1052條第1項第3款', 740),
 ('民法第1052條第1項第3款', 741),
 ({'民法第1052條第1項第3款'}, 743),
 ({'民法第1052條第2項'}, 745),
 ('民法第1052條第2項', 747),
 ('民法第1052條第1項第3款', 748),
 ('民法第1052條第1項第3款', 749),
 ('民法第1052條第2項', 750),
 ({'民法第1052條第1項第3款'}, 751),
 ('民法第1052條第1項第3款', 752),
 ('民法第1052條第1項第3款', 755),
 ('民法第1052條第1項第3款', 756),
 ({'民法第1052條第2項'}, 758),
 (None, 759),
 ({'民法第1052條第1項第3款'}, 760),
 ('民法第1052條第1項第2、3、5款', 764),
 ('民法第1052條第1項第3款', 767),
 ('民法第1052條第1項第3款', 768),
 ('民法第1052條第2項', 769),
 ('民法第1052條第2項', 770),
 ('民法第1052條第1項第3款', 772),
 ('民法第1052條第1項第3款', 773),
 ('民法第1052條第1項第3款', 775),
 ('民法第1052條第1項第3款', 776),
 ('民法第1052條第1項第3款', 777),
 ('民法第1052條第1項第3款', 778),
 ('民法第1052條第1項第3款', 779),
 ({'民法第1052條第2項'}, 780),
 ('民法第1052條第2

In [61]:
b = []
for i in range(len(final)):
    if final[i][0] == None:
        b.append(final[i][1])

In [62]:
len(b)

109

In [63]:
b

[759,
 810,
 1152,
 1837,
 2060,
 2113,
 2120,
 2147,
 2148,
 2150,
 2184,
 2192,
 2194,
 2253,
 2259,
 2265,
 2470,
 2474,
 2482,
 2489,
 2513,
 2528,
 2557,
 2598,
 2694,
 2706,
 2839,
 2856,
 2858,
 2886,
 2897,
 2921,
 2955,
 3049,
 3085,
 3124,
 3137,
 3171,
 3172,
 3173,
 3181,
 3234,
 3280,
 3376,
 3414,
 3421,
 3422,
 3423,
 3428,
 3437,
 3440,
 3441,
 3445,
 3446,
 3447,
 3448,
 3455,
 3459,
 3460,
 3466,
 3469,
 3471,
 3475,
 3476,
 3483,
 3485,
 3497,
 3504,
 3508,
 3510,
 3511,
 3513,
 3514,
 3515,
 3524,
 3525,
 3527,
 3528,
 3532,
 3534,
 3536,
 3544,
 3548,
 3549,
 3551,
 3552,
 3553,
 3558,
 3681,
 3773,
 3854,
 3910,
 3913,
 3916,
 3933,
 3954,
 3998,
 3999,
 4137,
 4196,
 4225,
 4243,
 4245,
 4248,
 4271,
 4272,
 4277,
 4292,
 4419]